In [5]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import recall_score,f1_score
dataset=pd.read_csv('D:\\student\\final_combined_dataset.csv')
#dataset=pd.read_csv('D:\\student\\merged.csv')
dataset_cols=list(dataset.columns)
print(dataset_cols)
maths_cols=['school','traveltime.x', 'studytime.x', 'failures.x', 'schoolsup.x','paid.x', 'activities.x', 'higher.x', 'romantic.x','freetime.x', 'goout.x', 'Dalc.x', 'Walc.x', 'health.x', 'absences.x','G1.x','G2.x','G3.x']
portugese_cols=['school','traveltime.y', 'studytime.y', 'failures.y', 'schoolsup.y','paid.y', 'activities.y', 'higher.y', 'romantic.y','freetime.y', 'goout.y', 'Dalc.y', 'Walc.y', 'health.y', 'absences.y','G1.y','G2.y','G3.y']
maths_dataset=dataset[maths_cols]
portugese_dataset=dataset[portugese_cols]
print(maths_dataset.shape)
print(portugese_dataset.shape)


['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu', 'Mjob', 'Fjob', 'reason', 'nursery', 'internet', 'guardian.x', 'traveltime.x', 'studytime.x', 'failures.x', 'schoolsup.x', 'famsup.x', 'paid.x', 'activities.x', 'higher.x', 'romantic.x', 'famrel.x', 'freetime.x', 'goout.x', 'Dalc.x', 'Walc.x', 'health.x', 'absences.x', 'G1.x', 'G2.x', 'G3.x', 'guardian.y', 'traveltime.y', 'studytime.y', 'failures.y', 'schoolsup.y', 'famsup.y', 'paid.y', 'activities.y', 'higher.y', 'romantic.y', 'famrel.y', 'freetime.y', 'goout.y', 'Dalc.y', 'Walc.y', 'health.y', 'absences.y', 'G1.y', 'G2.y', 'G3.y']
(882, 18)
(882, 18)


In [6]:
def encode_features(dataframe):#encode the nominal categorical features and convert marks to categorical
    cols=list(dataframe.columns)
    marks_cols=['G1.x','G2.x','G3.x','G1.y','G2.y','G3.y']
    for col in cols:
        if dataframe[col].dtype=='object':
            dataframe[col]=dataframe[col].astype('category')
            dataframe[col]=dataframe[col].cat.codes#label encoding
        if col in marks_cols:#marks are converted into two categories i.e binary classification
            marks=list(dataframe[col])
            fail=[]
            for m in marks:
                if m in list(range(0,10)):
                    fail.append(0)
                if m in list(range(10,21)):
                    fail.append(1)
            dataframe[col]=fail
    return dataframe
maths_encoded_dataset=encode_features(maths_dataset)
portugese_encoded_dataset=encode_features(portugese_dataset)

C:\Users\sudhe\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()


In [7]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
def predict_labels_normal(dataframe,model):#predict the class labels for each of the three grades for a given model
    index=-3
    data_cols=list(dataframe.columns)
    predicted_labels=[]
    try_folds=[2,3,4,5,6,7,8,9]
    for i in range(0,3):
        #complete_data=dataframe[data_cols[:index+1]]
        feature_data=dataframe[data_cols[:index]]
        label=dataframe[data_cols[index]]
        label_name=data_cols[index]
        f1_score_info={}
        for k in try_folds:
            trial_cv=KFold(n_splits=k,shuffle=True)
            rec_scores=[]
            for train_num,test_num in trial_cv.split(X=feature_data,y=label):
                train_indexes=list(train_num)#convert the indices to list
                test_indexes=list(test_num)
                train_features=feature_data.loc[train_indexes,:]#get all rows at specified positions
                test_features=feature_data.loc[test_indexes,:]
                train_labels=label.loc[train_indexes]
                test_labels=label.loc[test_indexes]
                #print('hi',train_features.shape,test_features.shape,train_labels.shape,test_labels.shape)
                model.fit(train_features,train_labels)
                preds=model.predict(test_features)
                r=f1_score(test_labels,preds,pos_label=0)
                rec_scores.append(r)
                #print(rec_scores)
            print('average f-1 score  for %r with %r folds marks is %r '% (label_name,k,sum(rec_scores)/len(rec_scores)))
            print('***********************************************************')
            f1_score_info[str(k)]=sum(rec_scores)/len(rec_scores)
        print('...............................................................')
        #print(recall_score_info)
        #recall_df=pd.DataFrame(recall_score_info,index=list(range(len(try_folds))))
        #print(recall_df.loc[0,:])
        index+=1
    return     
tr=RandomForestClassifier()
predict_labels_normal(maths_encoded_dataset,tr)
print('okay cool now for portugese subject')
predict_labels_normal(portugese_encoded_dataset,tr)

average f-1 score  for 'G1.x' with 2 folds marks is 0.6897735072065889 
***********************************************************
average f-1 score  for 'G1.x' with 3 folds marks is 0.7006363091316968 
***********************************************************
average f-1 score  for 'G1.x' with 4 folds marks is 0.6785108928026062 
***********************************************************
average f-1 score  for 'G1.x' with 5 folds marks is 0.6897498635222186 
***********************************************************
average f-1 score  for 'G1.x' with 6 folds marks is 0.6970595234983769 
***********************************************************
average f-1 score  for 'G1.x' with 7 folds marks is 0.7047814097390369 
***********************************************************
average f-1 score  for 'G1.x' with 8 folds marks is 0.6991627264083745 
***********************************************************
average f-1 score  for 'G1.x' with 9 folds marks is 0.7057279140612476 
****

In [8]:
from sklearn.model_selection import StratifiedKFold
def predict_labels_stratified(dataframe,model):#predict the class labels for each of the three grades for a given model
    index=-3
    data_cols=list(dataframe.columns)
    predicted_labels=[]
    try_folds=[2,3,4,5,6,7,8,9]
    for i in range(0,3):
        #complete_data=dataframe[data_cols[:index+1]]
        feature_data=dataframe[data_cols[:index]]
        label=dataframe[data_cols[index]]
        label_name=data_cols[index]
        f1_score_info={}
        for k in try_folds:
            trial_cv=StratifiedKFold(n_splits=k,shuffle=True)
            rec_scores=[]
            for train_num,test_num in trial_cv.split(X=feature_data,y=label):
                train_indexes=list(train_num)#convert the indices to list
                test_indexes=list(test_num)
                train_features=feature_data.loc[train_indexes,:]#get all rows at specified positions
                test_features=feature_data.loc[test_indexes,:]
                train_labels=label.loc[train_indexes]
                test_labels=label.loc[test_indexes]
                #print('hi',train_features.shape,test_features.shape,train_labels.shape,test_labels.shape)
                model.fit(train_features,train_labels)
                preds=model.predict(test_features)
                r=f1_score(test_labels,preds,pos_label=0)
                rec_scores.append(r)
                #print(rec_scores)
            print('average f-1 score  for %r with %r stratified folds marks is %r '% (label_name,k,sum(rec_scores)/len(rec_scores)))
            print('####################################################################')
            f1_score_info[str(k)]=sum(rec_scores)/len(rec_scores)
        print('.............................................................................')
        #print(recall_score_info)
        #recall_df=pd.DataFrame(recall_score_info,index=list(range(len(try_folds))))
        #print(recall_df.loc[0,:])
        index+=1
    return     
tr=RandomForestClassifier()
predict_labels_stratified(maths_encoded_dataset,tr)
print('cool...........')
predict_labels_stratified(portugese_encoded_dataset,tr)

average f-1 score  for 'G1.x' with 2 stratified folds marks is 0.6771428571428573 
####################################################################
average f-1 score  for 'G1.x' with 3 stratified folds marks is 0.6900229155649568 
####################################################################
average f-1 score  for 'G1.x' with 4 stratified folds marks is 0.7058828461842241 
####################################################################
average f-1 score  for 'G1.x' with 5 stratified folds marks is 0.7153682949457598 
####################################################################
average f-1 score  for 'G1.x' with 6 stratified folds marks is 0.7172341576723072 
####################################################################
average f-1 score  for 'G1.x' with 7 stratified folds marks is 0.6926130982926467 
####################################################################
average f-1 score  for 'G1.x' with 8 stratified folds marks is 0.6946955051619701 
#####